## 1.数据载入

In [438]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
#train.xlsx： 附件一 test.xlsx：附件二
DATA_PATH = 'data'
train = pd.read_excel('data/train.xlsx',index = False)
test = pd.read_excel('data/test.xlsx', index = False)

In [411]:
train.head()

In [412]:
test.head()

In [17]:
len(train), len(test)

(16016, 1489)

In [414]:
test.info()

In [413]:
train.info()

In [20]:
test.columns

Index(['编号', '任务id', '总里程', '业务类型', '需求类型1', '需求类型2', '线路编码', '是否续签', '始发网点',
       '目的网点', '计划发车时间', '计划发车日期', '分拨时间', '车辆长度', '车辆吨位', '标的展示策略', '打包类型',
       '子包号', '装卸的次数', '装的次数', '运输等级', '卸的次数', '是否需要装卸', '始发地省份名称', '车辆类型分类',
       '目的地省份名称', '交易对象', '计划卸货完成时间', '计划卸货等待时长', '计划靠车时间', '计划到达时间', '地区类型',
       '地区', '标的_创建时间', '标的_创建日期', '异常状态', '交易模式', '需求紧急程度'],
      dtype='object')

In [439]:
very_bad_cols = ['子包号','装卸的次数','装的次数','卸的次数','是否需要装卸'] #丢弃列
bad_cols_inTrain = ['C端议价最低价','B端议价最低价','调价审核完成时间']
part_Nan_cols = ['始发地省份名称','目的地省份名称','计划卸车等待时长']  #可以使用均值填充
target_cols = ['路线价格（不含税）','线路总成本']
id_cols = ['任务id']
redundant_cols = ['计划发车日期'] #内容重复
digital_cols = ['总里程','车辆长度','车辆吨位','计划卸货等待时长']
category_cols = ['业务类型','需求类型1','需求类型2','线路编码','是否续签','始发网点','目的网点','计划发车时间','计划发车日期',
                 '分拨时间','标的展示策略','打包类型','子包号','装卸的次数','装的次数','运输等级','卸的次数','是否需要装卸','始发地省份名称','车辆类型分类',
                 '目的地省份名称','交易对象','计划卸货完成时间','计划靠车时间','计划到达时间','地区类型','地区','标的_创建时间','标的_创建日期','异常状态',
                 '交易模式','需求紧急程度']
train_digital_cols = ['总里程','线路价格（不含税）','调价比例','线路指导价（不含税）','实际靠车时间','实际结束时间','实际发车时间','交易开始时间',
                      '交易结束时间','计划卸货完成时间','计划卸货等待时长','计划靠车时间','计划到达时间','地区','标的_创建时间','标的_创建日期','C端议价最低价',
                      'B端议价最低价','B端议价反馈数量','线路总成本','调价ECP创建时间','调价审核完成时间','最后一次询价时间','C端议价反馈数量','交易成功时长',
                      '交易成功时间','交易成功日期','计划发车时间','计划发车日期','分拨时间','实际到车时间']
# train_category_cols = 

## 2. 数据清洗

### 2.1 丢弃缺失值过多&多余的列

In [440]:
train = train.drop(columns = bad_cols_inTrain)
test = test.drop(columns = very_bad_cols)
train = train.drop(columns = very_bad_cols)
train = train.drop(columns = redundant_cols)
test = test.drop(columns = redundant_cols)

### 2.2 计算执行时间

In [476]:
print(type(train['线路编码']))
train['执行时间'] = train['线路编码'] - train['始发网点'] - train['目的网点']
#train_exe_time = DataFrame(train_exe_time,columns = '执行时间')
test['执行时间'] = test['线路编码'] - test['始发网点'] - test['目的网点']

train.info()
train.head()

### 2.3 时间戳

In [442]:
#标的_创建时间格式整理
ser1 = train['标的_创建日期'].astype(str)
ser2 = train['标的_创建时间'].astype(str)
train['标的_创建时间点'] = ser1.str.cat(ser2,sep = '  ').apply(pd.to_datetime,format='%Y-%m-%d  %H:%M:%S')
# train['标的_创建时间点'] = ser1.str.cat(ser2,sep = '  ')

ser3 = test['标的_创建日期'].astype(str)
ser4 = test['标的_创建时间'].astype(str)
test['标的_创建时间点'] = ser3.str.cat(ser4,sep = '  ').apply(pd.to_datetime,format='%Y-%m-%d  %H:%M:%S')

In [443]:
#将所有时间修改为时间戳
import time
##TRAIN
#for prediction
train['标的_创建时间点TS'] = train['标的_创建时间点'].apply(lambda x:x.timestamp())
train['实际到车时间TS'] = train['实际到车时间'].apply(lambda x:x.timestamp())
train['计划发车时间TS'] = train['计划发车时间'].apply(lambda x:x.timestamp())
train['计划靠车时间TS'] = train['计划靠车时间'].apply(lambda x:x.timestamp())
train['计划到达时间TS'] = train['计划到达时间'].apply(lambda x:x.timestamp())
train['实际到车时间TS'] = train['实际到车时间'].apply(lambda x:x.timestamp())
train['实际靠车时间TS'] = train['实际靠车时间'].apply(lambda x:x.timestamp())
train['实际结束时间TS'] = train['实际结束时间'].apply(lambda x:x.timestamp() if not pd.isnull(x) else 0) #缺失值
train['实际发车时间TS'] = train['实际发车时间'].apply(lambda x:x.timestamp())
train['交易开始时间TS'] = train['交易开始时间'].apply(lambda x:x.timestamp())
train['交易结束时间TS'] = train['交易结束时间'].apply(lambda x:x.timestamp())
train['分拨时间TS'] = train['分拨时间'].apply(lambda x:x.timestamp() if not pd.isnull(x) else 0) #缺失值
train['计划卸货完成时间TS'] = train['计划卸货完成时间'].apply(lambda x:x.timestamp() if not pd.isnull(x) else 0) #缺失值
#for evaluation
train['最后一次询价时间TS'] = train['最后一次询价时间'].apply(lambda x:x.timestamp() if not pd.isnull(x) else 0)
train['调价ECP创建时间TS'] = train['调价ECP创建时间'].apply(lambda x:x.timestamp() if not pd.isnull(x) else 0)
##TEST
test['计划发车时间TS'] = test['计划发车时间'].apply(lambda x:x.timestamp())
test['分拨时间TS'] = test['分拨时间'].apply(lambda x:x.timestamp())
test['计划卸货完成时间TS'] = test['计划卸货完成时间'].apply(lambda x:x.timestamp())
test['计划靠车时间TS'] = test['计划靠车时间'].apply(lambda x:x.timestamp())
test['计划到达时间TS'] = test['计划到达时间'].apply(lambda x:x.timestamp())
test['标的_创建时间点TS'] = test['标的_创建时间点'].apply(lambda x:x.timestamp())

### 2.4 缺失值处理

In [477]:
#time 均值填充
# train['实际结束时间TS'].fillna((train['实际结束时间TS'].mean()), inplace = True)
# train['计划卸货完成时间TS'].fillna((train['计划卸货完成时间TS'].mean()), inplace = True)
def FillNaTWithAve(name,df):
    nonZero = df[name].count() - (df[name] == 0).sum()
    mean = sum(df[name])/nonZero
    df[name][df[name]==0] = mean

FillNaTWithAve('实际结束时间TS', train)
FillNaTWithAve('计划卸货完成时间TS', train)
FillNaTWithAve('最后一次询价时间TS', train)
FillNaTWithAve('调价ECP创建时间TS', train)
FillNaTWithAve('分拨时间TS', train)

In [445]:
#categorical 按照种类最多的进行填补
#pack_type_col_dict = {'单边':0,'人工':1,'周期流向':2,'周期往返':3,}
train['打包类型'] = train['打包类型'].apply(lambda x:x if not pd.isnull(x) else '周期流向')
train['调价紧急程度'] = train['调价紧急程度'].apply(lambda x:x if not pd.isnull(x) else '常规')
train['始发地省份名称'] = train['始发地省份名称'].apply(lambda x:x if not pd.isnull(x) else '广东省')
train['目的地省份名称'] = train['目的地省份名称'].apply(lambda x:x if not pd.isnull(x) else '广东省')
train['交易模式'] = train['交易模式'].apply(lambda x:x if not pd.isnull(x) else '抢单')

test['始发地省份名称'] = test['始发地省份名称'].apply(lambda x:x if not pd.isnull(x) else '广东省')
test['目的地省份名称'] = test['目的地省份名称'].apply(lambda x:x if not pd.isnull(x) else '广东省')
#numerical 部分使用0填充
train['计划卸货等待时长'] = train['计划卸货等待时长'].apply(lambda x:x if not pd.isnull(x) else 0)

test['计划卸货等待时长'] = test['计划卸货等待时长'].apply(lambda x:x if not pd.isnull(x) else 0)

In [478]:
#丢弃部分训练样本
train.drop(np.where(np.isnan(train['线路总成本']))[0], axis = 0, inplace=True)
train.reset_index()

In [479]:
train.drop(columns = ['index'])

### 2.5 去除异常值

In [448]:
#交易成功时长
train = train.loc[((train['交易成功时长'] >= 1) & (train['交易成功时长'] < 45000))] #高斯分布去除1% outliers
#执行时长
train = train.loc[((train['执行时间'] > 0))]

## 3. 建立新特征

In [449]:
#执行时间的建立见2.2
#建立时间差特征
train['交易总时长'] = train['交易结束时间TS'] - train['交易开始时间TS']
train['司机工作总时长'] = train['实际结束时间TS'] - train['实际发车时间TS']
train['取货至目的地计划所用时间'] = train['计划到达时间TS'] - train['计划靠车时间TS']
train['标的-分拨时间差'] = train['标的_创建时间点TS'] - train['分拨时间TS']
train['发车到目的地计划所用时间'] = train['计划到达时间TS'] - train['计划发车时间TS']
train['计划卸货时间'] = train['计划卸货完成时间TS'] - train['计划到达时间TS']
train['标的-发车时间差'] = train['计划发车时间TS'] - train['标的_创建时间点TS']
train['发车-靠车时间差'] = train['计划发车时间TS'] - train['计划靠车时间TS']

test['发车到目的地计划所用时间'] = test['计划到达时间TS'] - test['计划发车时间TS']
test['标的-分拨时间差'] = test['标的_创建时间点TS'] - test['分拨时间TS']
test['计划卸货时间'] = test['计划卸货完成时间TS'] - test['计划到达时间TS']
test['取货至目的地计划所用时间'] = test['计划到达时间TS'] - test['计划靠车时间TS']
test['标的-发车时间差'] = test['计划发车时间TS'] - test['标的_创建时间点TS']
test['发车-靠车时间差'] = test['计划发车时间TS'] - test['计划靠车时间TS']

## 4.数据相关性分析

In [450]:

bad_cols_inTrain = ['C端议价最低价','B端议价最低价','调价审核完成时间']
part_Nan_cols = ['始发地省份名称','目的地省份名称','计划卸车等待时长']  #可以使用均值填充
target_cols = ['路线价格（不含税）','线路总成本','线路指导价（不含税）']
id_cols = ['任务id']

corr_cols = ['总里程','执行时间','线路编码','调价比例','线路指导价（不含税）','交易开始时间TS','交易结束时间TS',
             '计划卸货等待时长','计划靠车时间TS','计划到达时间TS','地区','标的_创建时间点TS', '线路总成本','交易成功时长',
                      '交易成功时间TS','计划发车时间TS','计划发车日期TS','分拨时间TS','实际到车时间TS','交易总时长',
            '司机工作总时长','取货至目的地计划所用时间','标的-分拨时间差', '发车到目的地计划所用时间', '计划卸货时间', '标的-发车时间差',
            '业务类型','需求类型1','需求类型2','是否续签','始发网点','目的网点','标的展示策略','打包类型','运输等级','始发地省份名称','车辆类型分类',
                 '目的地省份名称','交易对象','地区类型','地区', '异常状态', '交易模式','需求紧急程度']


#### 绘制相关性热力图

In [437]:
plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文字体设置-黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题
sns.set(font='SimHei',font_scale=8)  # 解决Seaborn中文显示问题并调整字体大小

figure, ax = plt.subplots(figsize=(100, 100))
data = pd.concat([train.drop(columns = ['异常状态','车辆类型分类']), test.drop(columns = ['编号'])]).corr() ** 2
#data = np.tril(data, k=-1)
data[data==0] = np.nan
sns.heatmap(np.sqrt(data), annot=False, cmap='viridis', ax=ax)

In [434]:
data = train.corrwith(train['线路指导价（不含税）']).agg('square')
data = data.drop(['线路价格（不含税）','线路总成本','线路指导价（不含税）'])
figure, ax = plt.subplots(figsize=(80, 10))
data.agg('sqrt').plot.bar(ax=ax, title = '线路指导价（不含税）影响因素')

In [433]:
data = train.corrwith(train['线路价格（不含税）']).agg('square')
data = data.drop(['线路价格（不含税）','线路总成本','线路指导价（不含税）'])
figure, ax = plt.subplots(figsize=(80, 10))
data.agg('sqrt').plot.bar(ax=ax, title = '线路价格（不含税）影响因素')

In [432]:
data = train.corrwith(train['线路总成本']).agg('square')
data = data.drop(['线路价格（不含税）','线路总成本','线路指导价（不含税）'])
figure, ax = plt.subplots(figsize=(80, 10))
data.agg('sqrt').plot.bar(ax=ax, title = '线路总成本影响因素')

### 影响调价比例因素
#### 用于二次、三次报价

In [431]:
# train['待接单时长'] = train['计划靠车时间TS'] - train['标的_创建时间点TS']
data = train.corrwith(train['调价比例']).agg('square')
data = data.drop(['调价比例'])
figure, ax = plt.subplots(figsize=(50, 10))
data.agg('sqrt').plot.bar(ax=ax, fontsize = 30)

In [430]:
plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文字体设置-黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题
sns.set(font='SimHei',font_scale=1.5)  # 解决Seaborn中文显示问题并调整字体大小
sns.set_style("whitegrid",{"font.sans-serif":['simhei', 'Arial']})

fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(10,5)) 

ax = sns.distplot(train['调价比例'],color = 'r', ax = ax)
x = ax.lines[0].get_xdata()
y = ax.lines[0].get_ydata()
maxid = np.argmax(y)
# low_quant = np.quantile(x,0.25,interpolation='lower')
# high_quant = np.quantile(x, 0.75, interpolation = 'higher')

ax.plot(x[maxid], y[maxid], 'bo', ms = 6)
ax.plot([x[7],x[54]], [0.1,0.1], 'bo', ms = 4)

### 交易成功时长、总里程、C端议价反馈数量、利润率、线路总成本分布
##### 用于评价定价

In [480]:
train['利润率'] = (train['线路价格（不含税）'] - train['线路总成本']) / train['线路总成本']
data = train.corrwith(train['交易成功时长']).agg('square')
data = data.drop(['交易成功时长'])
figure, ax = plt.subplots(figsize=(80, 10))
data.agg('sqrt').plot.bar(ax=ax, title = '交易成功时长影响因素')

In [341]:
train['利润率'] = (train['线路价格（不含税）'] - train['线路总成本']) / train['线路总成本']

In [ ]:
sns.set_style("whitegrid")
fig, ((ax1, ax2),(ax3,ax4), (ax5, ax6)) = plt.subplots(nrows=3,ncols=2,figsize=(30,12)) 
sns.set(font='SimHei',font_scale=2.5)
# sns.despine(left = 10000)

ax1 = sns.kdeplot(train['交易成功时长'], ax = ax1)

In [428]:
# from scipy.stats import norm

sns.set_style("whitegrid")
fig, ((ax1, ax2),(ax3,ax4), (ax5, ax6)) = plt.subplots(nrows=3,ncols=2,figsize=(30,12)) 
sns.set(font='SimHei',font_scale=2.5)
# sns.despine(left = 10000)

ax1 = sns.kdeplot(train['交易成功时长'], ax = ax1)
ax2 = sns.kdeplot(train['总里程'], ax = ax2)
ax3 = sns.kdeplot(train['C端议价反馈数量'], ax = ax3)
ax4 = sns.kdeplot(train['利润率'], ax = ax4)
ax5 = sns.kdeplot(train['车辆长度'], ax = ax5)
ax6 = sns.kdeplot(train['线路总成本'], ax = ax6)

train = train.drop(columns = '利润率')

In [427]:
x1 = ax1.lines[0].get_xdata()
y1 = ax1.lines[0].get_ydata()
maxid1 = np.argmax(y1)
print(x1[maxid1],y1[maxid1])

x2 = ax2.lines[0].get_xdata()
y2 = ax2.lines[0].get_ydata()
maxid2 = np.argmax(y2)
print(x2[maxid2],y2[maxid2])

x3 = ax3.lines[0].get_xdata()
y3 = ax3.lines[0].get_ydata()
maxid3 = np.argmax(y3)
print(x3[maxid3],y3[maxid3])

x4 = ax4.lines[0].get_xdata()
y4 = ax4.lines[0].get_ydata()
maxid4 = np.argmax(y4)
print(x4[maxid4],y4[maxid4])

x5 = ax5.lines[0].get_xdata()
y5 = ax5.lines[0].get_ydata()
maxid5 = np.argmax(y5)
print(x5[maxid5],y5[maxid5])

x6 = ax6.lines[0].get_xdata()
y6 = ax6.lines[0].get_ydata()
maxid6 = np.argmax(y6)
print(x6[maxid6],y6[maxid6])

print('最小值：')
print(np.max(x1), np.max(x2), np.max(x3), np.max(x4), np.max(x5), np.max(x6))
print('最大值：')
print(np.min(x1), np.min(x2), np.min(x3), np.min(x4), np.min(x5), np.min(x6))

In [371]:
train = train.drop(columns = '利润率')

## 5. 数据编码与归一化

In [452]:
prediction_cols = ['始发地省份名称', '分拨时间', '运输等级', '计划到达时间', '目的地省份名称', '标的_创建时间', 
                   '始发网点', '需求紧急程度', '交易模式', '卸的次数', '是否需要装卸', '计划靠车时间', '计划发车日期', '车辆长度', 
                   '装的次数', '是否续签', '交易对象', '车辆吨位', '目的网点', '异常状态', '计划发车时间', '车辆类型分类', '任务id',
                   '需求类型2', '总里程', '打包类型', '标的展示策略', '业务类型', '计划卸货等待时长', '地区', '标的_创建日期', '线路编码', 
                   '计划卸货完成时间', '需求类型1', '地区类型', '子包号']
#目的网点 始发网点 看作digital
digital_cols = ['总里程', '线路编码', '始发网点', '目的网点', '车辆长度', '车辆吨位','计划卸货等待时长',
               '执行时间','计划发车时间TS', '分拨时间TS', '计划卸货完成时间TS','计划靠车时间TS', '计划到达时间TS',
                '标的_创建时间点TS', '发车到目的地计划所用时间', '标的-分拨时间差',
       '计划卸货时间', '取货至目的地计划所用时间', '标的-发车时间差', '发车-靠车时间差']
category_cols_onehot = ['业务类型A','业务类型B','需求类型1A','需求类型1B','需求类型1C','需求类型2A','需求类型2B','是否续签A','是否续签B','是否续签C','是否续签D',
                        '打包类型A','打包类型B','打包类型C','打包类型D','运输等级A','运输等级B','运输等级C','始发地省份名称A','始发地省份名称B',
                        '始发地省份名称C','始发地省份名称D','车辆类型分类', '目的地省份名称A','目的地省份名称B','目的地省份名称C','目的地省份名称D','交易对象A','交易对象B','交易对象C',
                        '地区类型A','地区类型B','地区A','地区B','地区C','地区D','地区E','地区F', '异常状态', '交易模式A','交易模式B','交易模式C','需求紧急程度A','需求紧急程度B','需求紧急程度C']
#目的网点 始发网点 看作category
category_cols = ['业务类型','需求类型1','需求类型2','是否续签','打包类型','运输等级','始发地省份名称','车辆类型分类',
                 '目的地省份名称','交易对象', '地区类型','地区', '异常状态', '交易模式','需求紧急程度']

train_digital_cols = ['总里程','线路价格（不含税）','调价比例','线路指导价（不含税）','实际靠车时间','实际结束时间','实际发车时间','交易开始时间',
                      '交易结束时间','计划卸货完成时间','计划卸货等待时长','计划靠车时间','计划到达时间','地区','标的_创建时间','标的_创建日期','C端议价最低价',
                      'B端议价最低价','B端议价反馈数量','线路总成本','调价ECP创建时间','调价审核完成时间','最后一次询价时间','C端议价反馈数量','交易成功时长',
                      '交易成功时间','交易成功日期','计划发车时间','计划发车日期','分拨时间','实际到车时间']

### Digital

In [453]:
from sklearn.preprocessing import MinMaxScaler

sacalar = MinMaxScaler()
train_digital = sacalar.fit_transform(train[digital_cols])
test_digital = sacalar.transform(test[digital_cols])

### Categorical

In [483]:
test['交易模式'] = test['交易模式'].astype(str)
train['交易模式'] = train['交易模式'].astype(str)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

train_category, test_category = None, None
drop_cols = [ ]
for col in [var for var in category_cols if var not in drop_cols]:
    lbe, ohe = LabelEncoder(), OneHotEncoder()
    
    lbe.fit(pd.concat([train[col], test[col]]).values.reshape(-1, 1))
    train[col] = lbe.transform(train[col])
    test[col] = lbe.transform(test[col])
    
    ohe.fit(pd.concat([train[col], test[col]]).values.reshape(-1, 1))
    oht_train = ohe.transform(train[col].values.reshape(-1, 1)).todense()
    oht_test = ohe.transform(test[col].values.reshape(-1, 1)).todense()
    
    if train_category is None:
        train_category = oht_train
        test_category = oht_test
    else:
        train_category = np.hstack((train_category, oht_train))
        test_category = np.hstack((test_category, oht_test))

In [455]:
train_digital.shape, test_digital.shape, train_category.shape, test_category.shape

((15201, 20), (1489, 20), (15201, 45), (1489, 45))

In [456]:
train_features = np.hstack((train_digital, train_category))
test_features = np.hstack((test_digital, test_category))
train_features.shape, test_features.shape

((15201, 65), (1489, 65))

In [457]:
target_cols = ['线路价格（不含税）','线路总成本','线路指导价（不含税）']
train_labels1 = train[target_cols[0]]
train_labels2 = train[target_cols[1]]
train_labels3 = train[target_cols[2]]
train_labels1.shape

(15201,)

## 6. 在验证集上预测
#### (线路价格:train_labels1 线路总成本: train_labels2)

In [485]:
#数据划分，10%为验证集
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_features, train_labels3, test_size=0.10, random_state=42)

### 主成分分析降维

In [231]:
from sklearn.decomposition import PCA
#pca = PCA(n_components = 'mle')
pca = PCA(n_components = 15)
X_val_PCA_new = pca.fit_transform(X_val)
train_features_PCA_new = pca.fit_transform(train_features)
test_features_PCA_new = pca.fit_transform(test_features)
X_train_PCA_new = pca.fit_transform(X_train)
print('保留的特征数：')
print(pca.n_components_)
print('特征所占比重分别为：')
print(pca.explained_variance_ratio_)
print('特征所占比重之和：')
print(sum(pca.explained_variance_ratio_))
print("新的特征为:")
print(X_train_new.shape)

保留的特征数：
15
特征所占比重分别为：
[0.30838612 0.15774897 0.10853085 0.09251935 0.07850997 0.06415782
 0.04841064 0.03045264 0.02433734 0.01998103 0.0151864  0.01361913
 0.00765552 0.00625841 0.00570833]
特征所占比重之和：
0.9814625268835183
新的特征为:
(13680, 15)


### Xgboost

In [373]:
#Best for label2
import xgboost as xgb
from sklearn import metrics
#dataset
dtrain = xgb.DMatrix(X_train, label = y_train)

#parameters
num_round = 300
param = {'bst:max_depth':3, 'bst:eta':1, 'silent':1, 'objective':'reg:squarederror', 'nthread':2 }
plst = param.items()

#fit the model
bst = xgb.train(param, dtrain, num_round)
#save the model
bst.save_model('0001.model')

dtest = xgb.DMatrix(X_val)
ypred1 = bst.predict(dtest)

val_MSE = metrics.mean_squared_error(y_val, ypred1) #验证集上的auc值
print('MSE:',val_MSE)

from sklearn.metrics import r2_score
r2 = r2_score(y_val, ypred1)
print('R-Square:',r2)

[16:14:03] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { bst:eta, bst:max_depth, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


MSE: 1049.3706541477445
R-Square: 0.9998780641596996


In [460]:
#Best for label1
import xgboost as xgb
from sklearn import metrics
#dataset
dtrain = xgb.DMatrix(X_train, label = y_train)

#parameters
num_round = 300
param = {'bst:max_depth':3, 'bst:eta':1, 'silent':1, 'objective':'reg:squarederror', 'nthread':2 }
plst = param.items()

#fit the model
bst = xgb.train(param, dtrain, num_round)
#save the model
bst.save_model('0001.model')

dtest = xgb.DMatrix(X_val)
ypred1 = bst.predict(dtest)

val_MSE = metrics.mean_squared_error(y_val, ypred1) #验证集上的auc值
print('MSE:',val_MSE)

from sklearn.metrics import r2_score
r2 = r2_score(y_val, ypred1)
print('R-Square:',r2)

[22:38:28] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { bst:eta, bst:max_depth, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


MSE: 39866.3669650506
R-Square: 0.9974878166411488


### 线性回归

In [393]:
#labels2
from sklearn.linear_model import LinearRegression

clf_after = LinearRegression()
clf_after.fit(X_train, y_train)

ypred2 = clf_after.predict(X_val)
ypred2.shape
test_MSE = metrics.mean_squared_error(y_val,ypred)#验证集上的auc值
print('MSE:',test_MSE)
from sklearn.metrics import r2_score
r2 = r2_score(y_val, ypred2)
print('R-Square:',r2)

MSE: 943.1033187924988
R-Square: 0.995927306217744


In [461]:
#labels1
from sklearn.linear_model import LinearRegression

clf_after = LinearRegression()
clf_after.fit(X_train, y_train)

ypred2 = clf_after.predict(X_val)
ypred2.shape
test_MSE = metrics.mean_squared_error(y_val,ypred)#验证集上的auc值
print('MSE:',test_MSE)
from sklearn.metrics import r2_score
r2 = r2_score(y_val, ypred2)
print('R-Square:',r2)

MSE: 1527980.0858834076
R-Square: 0.9823368036358394


### 随机森林

In [377]:
#Best for label2
from sklearn.ensemble import RandomForestRegressor
# grid search for the best parameters
max_features = {0.2,0.3,0.35,0.4}
n_estimators = {80,90,100,120}
rf = RandomForestRegressor(random_state=0,max_features = 0.35, n_estimators = 90)
rf.fit(X_train,y_train)
ypred3 = rf.predict(X_val)
test_MSE = metrics.mean_squared_error(y_val,ypred3)#验证集上的auc值
print('MSE:',test_MSE)
from sklearn.metrics import r2_score
r2 = r2_score(y_val, ypred3)
print('R-Square:',r2)

MSE: 855.3593863610247
R-Square: 0.9999006080786398


In [467]:
#Best for label1 0.2, 100
from sklearn.ensemble import RandomForestRegressor
# grid search for the best parameters
max_features = {0.2,0.3,0.35,0.4}
n_estimators = {80,90,100,120}
rf = RandomForestRegressor(random_state=0,max_features = 0.3, n_estimators =100)
rf.fit(X_train,y_train)
ypred3 = rf.predict(X_val)
test_MSE = metrics.mean_squared_error(y_val,ypred3)#验证集上的auc值
print('MSE:',test_MSE)
from sklearn.metrics import r2_score
r2 = r2_score(y_val, ypred3)
print('R-Square:',r2)

MSE: 29766.57068474688
R-Square: 0.9981242563790715


## Ensemble Model 集成模型

In [376]:
#Ensemble lable2(成本价)
ypred = 0.05*ypred1 + 0.05*ypred2 + 0.9*ypred3
test_MSE = metrics.mean_squared_error(y_val,ypred)#验证集上的auc值
print('MSE:',test_MSE)
from sklearn.metrics import r2_score
r2 = r2_score(y_val, ypred)
print('R-Square:',r2)

MSE: 943.1033187924988
R-Square: 0.9998904123197914


In [470]:
#Ensemble lable1(指导价)
ypred = 0.2*ypred1 + 0.1*ypred2 + 0.7*ypred3
test_MSE = metrics.mean_squared_error(y_val,ypred)#验证集上的auc值
print('MSE:',test_MSE)
from sklearn.metrics import r2_score
r2 = r2_score(y_val, ypred)
print('R-Square:',r2)

MSE: 30736.33815109412
R-Square: 0.998063146378929


## 利用模型预测并保存结果

In [471]:
#用全部样本训练模型:总成本 RF
#Best for label2
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=0,max_features = 0.35, n_estimators = 90)
rf.fit(train_features, train_labels2)
total_cost = rf.predict(test_features)

In [474]:
#用全部样本训练模型:定价 label1 RF
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=0,max_features = 0.35, n_estimators = 90)
rf.fit(train_features, train_labels1)
first_price = rf.predict(test_features)

In [484]:
First_price = pd.Series(first_price,name = '预测一次报价')
Total_cost = pd.Series(total_cost, name = '预测总成本')
First_price.to_csv('First_price.csv')
Total_cost.to_csv('Total_cost.csv')

### 利用随机森林模型探索特征重要性

In [425]:
#总成本
names = ['总里程', '线路编码', '始发网点', '目的网点', '车辆长度', '车辆吨位','计划卸货等待时长',
               '执行时间','计划发车时间TS', '分拨时间TS', '计划卸货完成时间TS','计划靠车时间TS', '计划到达时间TS',
                '标的_创建时间点TS', '发车到目的地计划所用时间', '标的-分拨时间差',
       '计划卸货时间', '取货至目的地计划所用时间', '标的-发车时间差', '发车-靠车时间差','业务类型A','业务类型B','需求类型1A','需求类型1B','需求类型1C',
         '需求类型2A','需求类型2B','是否续签A','是否续签B','是否续签C','是否续签D',
                        '打包类型A','打包类型B','打包类型C','打包类型D','运输等级A','运输等级B','运输等级C','始发地省份名称A','始发地省份名称B',
                        '始发地省份名称C','始发地省份名称D','车辆类型分类', '目的地省份名称A','目的地省份名称B','目的地省份名称C','目的地省份名称D','交易对象A','交易对象B','交易对象C',
                        '地区类型A','地区类型B','地区A','地区B','地区C','地区D','地区E','地区F', '异常状态', '交易模式A','交易模式B','交易模式C','需求紧急程度A','需求紧急程度B','需求紧急程度C']
Feature_importances = sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), names), reverse=True)
print(Feature_importances)

feature_imp_values = sorted(rf.feature_importances_, reverse = True)
feature_imp_values
Feature_imp_Series = pd.Series(feature_imp_values, index = names)
plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文字体设置-黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题
sns.set(font='SimHei',font_scale=1.5)  # 解决Seaborn中文显示问题并调整字体大小
sns.set_style("whitegrid",{"font.sans-serif":['simhei', 'Arial']})
fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(21,4)) 

Feature_imp_Series.plot.bar(ax=ax, title = '特征重要性')


In [ ]:
#指导价
names = ['总里程', '线路编码', '始发网点', '目的网点', '车辆长度', '车辆吨位','计划卸货等待时长',
               '执行时间','计划发车时间TS', '分拨时间TS', '计划卸货完成时间TS','计划靠车时间TS', '计划到达时间TS',
                '标的_创建时间点TS', '发车到目的地计划所用时间', '标的-分拨时间差',
       '计划卸货时间', '取货至目的地计划所用时间', '标的-发车时间差', '发车-靠车时间差','业务类型A','业务类型B','需求类型1A','需求类型1B','需求类型1C',
         '需求类型2A','需求类型2B','是否续签A','是否续签B','是否续签C','是否续签D',
                        '打包类型A','打包类型B','打包类型C','打包类型D','运输等级A','运输等级B','运输等级C','始发地省份名称A','始发地省份名称B',
                        '始发地省份名称C','始发地省份名称D','车辆类型分类', '目的地省份名称A','目的地省份名称B','目的地省份名称C','目的地省份名称D','交易对象A','交易对象B','交易对象C',
                        '地区类型A','地区类型B','地区A','地区B','地区C','地区D','地区E','地区F', '异常状态', '交易模式A','交易模式B','交易模式C','需求紧急程度A','需求紧急程度B','需求紧急程度C']
Feature_importances = sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), names), reverse=True)
print(Feature_importances)

feature_imp_values = sorted(rf.feature_importances_, reverse = True)
feature_imp_values
Feature_imp_Series = pd.Series(feature_imp_values, index = names)
plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文字体设置-黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题
sns.set(font='SimHei',font_scale=1.5)  # 解决Seaborn中文显示问题并调整字体大小
sns.set_style("whitegrid",{"font.sans-serif":['simhei', 'Arial']})
fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(21,4)) 

Feature_imp_Series.plot.bar(ax=ax, title = '特征重要性')

## 7. 基于历史概率分布拟合的定价规则

### 归一化后的影响因素分布 目的网点、执行时间

In [87]:
# wait_Time = (train['待接单时长'] - train['待接单时长'].mean()) / train['待接单时长'].std()
Des_net =  (train['目的网点'] - train['目的网点'].mean()) / train['目的网点'].std()
exe_Time = (train['执行时间'] - train['执行时间'].mean()) / train['执行时间'].std()

In [424]:
sns.set_style("whitegrid")
fig, (ax1, ax2) = plt.subplots(nrows=1,ncols=2,figsize=(20,6)) 
sns.set(font='SimHei',font_scale=1)
# sns.despine(left = 10000)
ax1 = sns.kdeplot(exe_Time, ax = ax1)
ax2 = sns.kdeplot(Des_net, ax = ax2)

### 使用CFTool 拟合的多项式模型

In [320]:
Des_net =  train['目的网点']
exe_Time = train['执行时间']
#三次报价计算（加入噪声） f(D,T)
def third_price(D,T,length, mu1, mu2, sigma1, sigma2, per):
    #单位向量
    i_original = []
    for i in range(0,length): i_original.append(1)
    I = pd.Series(i_original)
    d1 = np.random.normal(loc=mu1,scale=sigma1,size=length) # mean : loc std :  scale num : size
    d2 = np.random.normal(loc = mu2, scale = sigma2, size = length) # deviations
    p = [1.837,-0.2201,0.02774,0.007887,0.005703,-0.001838,-0.0001485,0,0]
    DT = [I, D,T, D**2, D*T, T**2, T*D**2,D*T**2, T**3]
    third_price = np.dot(p,DT) + per * d1 + (1 - per) * d2
    return third_price
#  F(D, T)
third_price = third_price(Des_net, exe_Time, len(Des_net), 10,1.9,0.08,0.001,0)
Third_price = []
for i in range(len(third_price)):
    if third_price[i] < 0:
        Third_price.append(1.1 - third_price[i])
    else:
        Third_price.append(third_price[i])
TPrice = pd.Series(Third_price, name = '调价比例拟合分布')

In [423]:
#比较前后的分布
sns.set_style("whitegrid")
fig, (ax1,ax2) = plt.subplots(nrows=1,ncols=2,figsize=(16,6)) 
sns.set(font='SimHei',font_scale=1)
# sns.despine(left = 10000)
ax1 = sns.kdeplot(TPrice/2.2, ax = ax1)
ax2 = sns.kdeplot(train['调价比例'],ax = ax2)

### 计算附件2中的三次报价

In [486]:
Des_net =  test['目的网点']
exe_Time = test['执行时间']
#三次报价计算（加入噪声） f(D,T)
def third_price(D,T,length, mu1, mu2, sigma1, sigma2, per):
    #单位向量
    i_original = []
    for i in range(0,length): i_original.append(1)
    I = pd.Series(i_original)
    d1 = np.random.normal(loc=mu1,scale=sigma1,size=length) # mean : loc std :  scale num : size
    d2 = np.random.normal(loc = mu2, scale = sigma2, size = length) # deviations
    p = [1.837,-0.2201,0.02774,0.007887,0.005703,-0.001838,-0.0001485,0,0]
    DT = [I, D,T, D**2, D*T, T**2, T*D**2,D*T**2, T**3]
    third_price = np.dot(p,DT) + per * d1 + (1 - per) * d2
    return third_price
#  F(D, T)
third_price = third_price(Des_net, exe_Time, len(Des_net), 10,1.9,0.08,0.001,0)
Third_price = []
for i in range(len(third_price)):
    if third_price[i] < 0:
        Third_price.append((1.1 - third_price[i])/2.2)
    else:
        Third_price.append(third_price[i]/2.2)
TPrice = pd.Series(Third_price, name = '调价比例拟合分布')

TPrice_real = TPrice * Total_cost

In [492]:
TPrice_real.to_csv('Third_price.csv')
SPrice_real = TPrice_real*0.8
SPrice_real.to_csv('Second_price.csv')

#### 第二次、三次报价规整

In [488]:
Price_reg = pd.read_excel('data/附件3：计算结果new.xlsx', index = False)

In [491]:
for i in range(len(Price_reg['任务id'])):
    if Price_reg['一次报价'][i] > Price_reg['二次报价'][i] and Price_reg['一次报价'][i] < Price_reg['三次报价'][i]:
        Price_reg['二次报价'][i] = Price_reg['三次报价'][i]
    if Price_reg['一次报价'][i] > Price_reg['二次报价'][i]:
        Price_reg['二次报价'][i] = Price_reg['一次报价'][i]
        Price_reg['三次报价'][i] = Price_reg['一次报价'][i]

In [490]:
Price_reg.to_csv('计算结果new.csv')